In [12]:
import os
import subprocess
import MDAnalysis as mda
import numpy as np

In [ ]:
# Extract the suitable strfiles
# Data
data_path = '/home/ys/SongYang/GoMartini3/gbp_sw/Martini3_300ns/Data/repeat-48'
strfile = os.path.join(data_path, "md_start.pdb")
trjfile_processed = os.path.join(data_path, 'md_rottrans.xtc')
trjfile_raw = os.path.join(data_path, "md_whole.xtc")

# Extracting Process
u_processed = mda.Universe(strfile, trjfile_processed)
u_raw = mda.Universe(strfile, trjfile_raw)

# Calculae the distance array
def Distance_analysis(u):
    sel1 = u.select_atoms('name BB and resid 59')
    sel2 = u.select_atoms('name BB and resid 130')
    data = []
    for ts in u.trajectory:
        dist = np.linalg.norm(sel1.positions-sel2.positions, axis=1)[0]
        data.append([ts.frame, ts.time, dist])

    return np.array(data)

dist_array = Distance_analysis(u_processed)
dist_min = dist_array[:,2].min()
dist_max = dist_array[:,2].max()

# Get the frame according to the distance
frames_sel = []
for dist in np.linspace(dist_min, dist_max, 10):
    frame_sel = np.abs(dist_array[:,2] - dist).argmin()
    frames_sel.append(frame_sel)
    print(frame_sel, dist_array[frame_sel, 2], dist)

# Write the gro file
for i, frame in enumerate(frames_sel):
    u_raw.trajectory[frame]
    u_raw.atoms.write(f'GlnBP_No{i}.gro')

In [20]:
def NPTProcess(strfile, top='system.top', mdp='npt.mdp'):
    result = subprocess.run(f"""
    gmx grompp -f em.mdp -c {strfile} -p system_open.top -o em.tpr
    gmx mdrun -deffnm em -v  -nt 8

    gmx grompp -f npt.mdp -c em.gro -p system_open.top -o npt.tpr -r em.gro
    gmx mdrun -deffnm npt -v -nt 8

    echo 1 0|gmx trjconv -s npt.tpr -f npt.gro -o {strfile} -pbc mol -center
    rm *.edr em.gro  npt.gro *.log *.tpr *.trr *.cpt  *.xtc *.pdb npt.gro em.gro mdout.mdp
    rm *#
                """, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    if result.returncode == 0:
        print(f"{strfile} finished!")
        pass
    else:
        print(f"Error: {strfile} Return code: {result.returncode}")
        print("Standard Output:")
        print(result.stdout.decode())
        print("Standard Error:")
        print(result.stderr.decode())

In [21]:
# Set velocity (NPT process)
for i, frame in enumerate(frames_sel):
    NPTProcess(f'GlnBP_No{i}.gro', top='system.top', mdp='npt.mdp')


